In [1]:
import pandas as pd
import numpy as np
import os
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from natsort import natsorted
from scipy.stats import pearsonr
import seaborn as sns
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import matplotlib.cm as cm
from pygam import LinearGAM, s, f
from scipy import stats
sns.set(font_scale=1.5, style="white")

In [2]:
regionname = pd.read_csv('../../Dataset/childhood/atlas/schaefer400_regions.csv')
sarank = pd.read_csv('../../Dataset/childhood/saaxis/Sensorimotor_Association_Axis_AverageRanks.csv')
sarank = pd.concat([regionname.reset_index(drop=True), sarank.reset_index(drop=True)],axis=1)
sarank = sarank.loc[:, ['name', 'finalrank.wholebrain']]

In [ ]:
# read expression data
data_dir = '../../Dataset/childhood/rna'
gene_columns = pd.read_csv(os.path.join(data_dir, 'columns_metadata.csv'))
gene_rows = pd.read_csv(os.path.join(data_dir, 'rows_metadata.csv'))
gene_columns['id'] = range(1, len(gene_columns) + 1)
expr_matrix = pd.read_csv(os.path.join(data_dir, 'expression_matrix.csv'), header=None)
expr_matrix = expr_matrix.iloc[:, 1:] 

In [ ]:
# remove genes without an entrez id
expr_subset = expr_matrix[~gene_rows['entrez_id'].isna()]
rows_subset = gene_rows[~gene_rows['entrez_id'].isna()]

In [ ]:
# log2 transform expression data and collapse duplicated transcripts
log2_expr = np.log2(expr_subset + 1)
log2_expr['entrez_id'] = rows_subset['entrez_id']

In [ ]:
# select inhibitory-related markers
markers  = ['PVALB', 'SST', 'VIP', 'GAD1', 'GAD2'] 
for marker in markers:
    ids = gene_rows[gene_rows.gene_symbol==marker].entrez_id.values[0]
    df_marker = log2_expr[log2_expr.entrez_id==ids].iloc[:, :-1].T.values
    gene_columns[marker] = df_marker

In [ ]:
sa_cortex = np.unique([x for x in gene_columns.structure_name if 'cortex' in x])
sa_cortex = pd.DataFrame([x for x in sa_cortex if 'cereb' not in x])
sa_cortex.columns = ['structure_name']
sa_cortex['sa'] = [7,10,8,-1,6,-1,4,5,1,3,-1,2,0,-1,9]
gene_columns = pd.merge(gene_columns, sa_cortex, on='structure_name', how='left')

In [ ]:
age = pd.DataFrame(gene_columns.age.unique())
age['rank'] = np.arange(1,age.shape[0]+1)
age.columns = ['age', 'rank']
gene_columns = pd.merge(gene_columns, age, on='age', how='left')

In [ ]:
adults = [ '18 yrs', '19 yrs', '21 yrs','23 yrs']
cbpd_wide = ['3 yrs','4 yrs', '8 yrs', '11 yrs','13 yrs']

In [ ]:
wide = gene_columns.loc[[x for x in gene_columns.index if gene_columns.loc[x,'age'] in adults], :]
wide = wide[wide.sa>-1].sort_values('sa')

In [ ]:
my_palette = {c: sns.color_palette('PuOr', n_colors=(11)).as_hex()[c] for c in np.arange(11)}
for j,sets in enumerate([[0,1,2],[3,4]]):
    gene_columns = gene_columns[gene_columns.sa>-1]
    for i, marker in enumerate([markers[x] for x in sets]):
        figure = plt.figure()
        wide = gene_columns.loc[[x for x in gene_columns.index if gene_columns.loc[x,'age'] in cbpd_wide], :]
        wide = wide[wide.age!='4 yrs']
        for t,structure_name in enumerate(wide.sa.unique()[:-1]):
            wide_st = wide[wide.sa==structure_name]
            wide_st = wide_st.loc[:, ['age', 'sa', 'rank', 'donor_id', 'PVALB', 'SST', 'VIP', 'GAD1', 'GAD2']]
            donor = wide_st.groupby(['rank', 'age']).std().reset_index().sort_values('rank')
            wide_st = wide_st.groupby(['age']).mean().reset_index().sort_values('rank')
            x = [str(t) for t in wide_st['rank'].values]
            y = wide_st[marker].values
            yerr = donor[marker].values
            plt.errorbar(x,y , yerr=yerr, color=my_palette[structure_name]) 
            if j == 0:
                plt.ylim([2,9])
            else:
                plt.ylim([1.5,5])
            plt.ylabel('Marker expression (z-score)')
            plt.xlabel('Age')
            plt.legend([wide[wide.sa==x].structure_acronym.values[0] for x in wide.sa.unique()], bbox_to_anchor=(1.05, 1), loc='upper left')

In [12]:
wide.to_csv('RNA.csv', index=False)

In [14]:
wide.age.unique()

array(['3 yrs', '8 yrs', '11 yrs', '13 yrs'], dtype=object)